In [2]:
!pip install transformers datasets sentencepiece
!pip install --upgrade huggingface_hub transformers
!pip install -q --upgrade transformers

In [9]:
import torch
import pandas as pd
import re
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
data = pd.read_excel('/content/drive/MyDrive/TripAdvisor datasets/MeIA_2025_train.xlsx', sheet_name="MeIA_2025_train")
data = data.drop(columns=['Town','Region','Type'])
data = data.rename(columns={'Polarity': 'labels'})
data = data.rename(columns={'Review': 'review'})
data['labels'] = data['labels']

print("Dimensiones del dataset de entrenamiento:", data.shape)
data.head()

Dimensiones del dataset de entrenamiento: (5000, 2)


,review,labels
0,Un Restaurante te invita por su ambiente tan a...,2.0
1,Pagamos 25 pesos por la entrada y no es gran c...,3.0
2,Mi esposa y yo nos alojamos en el Dreams por 4...,3.0
3,"La única decepción puede no ser José Cuervo, p...",2.0
4,Cuando leí los comentarios sobre cómo son las ...,1.0


In [40]:
def limpieza_minima(texto):
    texto = re.sub(r"http\S+", "", texto)         # URLs
    #texto = re.sub(r"[^\x00-\x7F]+", " ", texto)  # Caracteres raros si los hay

    # Quitar emojis (rango unicode de emojis)
    texto = re.sub(r'[\U00010000-\U0010ffff]', '', texto)

    # Quitar asteriscos, guiones, comillas raras, comillas simples repetidas, etc.
    texto = re.sub(r'[*“”"\'´`]', '', texto)

    # Quitar saltos de línea y tabs
    texto = texto.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')

    # Quitar múltiples espacios
    texto = re.sub(r'\s+', ' ', texto)

    # Eliminar espacios al inicio y final
    texto = texto.strip()

    # minusculas
    texto = texto.lower()
    return texto



In [41]:
# limpiar data
data['review'] = data['review'].apply(limpieza_minima)
data.head()

,review,labels
0,un restaurante te invita por su ambiente tan a...,2.0
1,pagamos 25 pesos por la entrada y no es gran c...,3.0
2,mi esposa y yo nos alojamos en el dreams por 4...,3.0
3,"la única decepción puede no ser josé cuervo, p...",2.0
4,cuando leí los comentarios sobre cómo son las ...,1.0


In [5]:
import torch
print("¿GPU disponible?:", torch.cuda.is_available())
print("Nombre de GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


¿GPU disponible?: True
Nombre de GPU: Tesla T4


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# Tokeniza todo el dataset
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True, padding="max_length", max_length=256)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
from datasets import Dataset

dataset = Dataset.from_pandas(data[["review", "labels"]])
dataset = dataset.class_encode_column("labels")
dataset = dataset.train_test_split(test_size=0.15)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Stringifying the column:   0%|          | 0/5000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4250 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from huggingface_hub import login

model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=5)
model.to("cuda")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average='weighted')
    }


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

trainer.train() # Requiere WANB

trainer.save_model("/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto")  # Carpeta donde guardar
tokenizer.save_pretrained("/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-9-170297371.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayelaldo (ayelaldo-unam-universidad-nacional-aut-noma-de-m-xico) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.073700,0.997828,0.552000,0.543836
2,0.889900,0.973577,0.566667,0.553497
3,0.675900,1.063330,0.568000,0.573572
4,0.462100,1.295893,0.546667,0.548265
5,0.275100,1.564689,0.561333,0.563168


('/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto/tokenizer_config.json',
 '/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto/special_tokens_map.json',
 '/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto/vocab.txt',
 '/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto/added_tokens.json',
 '/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto/tokenizer.json')

In [10]:
trainer.evaluate()

{'eval_loss': 1.0633304119110107,
 'eval_accuracy': 0.568,
 'eval_f1': 0.5735724348433426,
 'eval_runtime': 10.1306,
 'eval_samples_per_second': 74.033,
 'eval_steps_per_second': 4.639,
 'epoch': 5.0}

In [12]:
df_eval = pd.read_excel('/content/drive/MyDrive/TripAdvisor datasets/MeIA_2025_test_two_labels.xlsx', sheet_name="MeIA_2025_test_wo_labels")
df_eval["review"] = df_eval["Review"].apply(limpieza_minima)
df_eval = df_eval.drop(columns=['Town','Region','Type'])#solo si quieres quitar estas columnas

In [13]:
df_eval

,ID,Review,review
0,0,Nos alojamos 5 noches en el Coco Beach Bungalo...,nos alojamos 5 noches en el coco beach bungalo...
1,1,"Es lindo, pero me hubiera gustado que nos come...","es lindo, pero me hubiera gustado que nos come..."
2,2,"El lugar es muy bonito, lo tienen MUY descuida...","el lugar es muy bonito, lo tienen muy descuida..."
3,3,"Además de que Taxco es mágico, el zócalo tiene...","además de que taxco es mágico, el zócalo tiene..."
4,4,"Sin duda un lugar Mágico para visitar, debes i...","sin duda un lugar mágico para visitar, debes i..."
...,...,...,...
2495,2495,es un restaurante muy atractivo en una extensi...,es un restaurante muy atractivo en una extensi...
2496,2496,"Así es , lamento decirlo pero los originales 5...","así es , lamento decirlo pero los originales 5..."
2497,2497,La vista es mejor que el servicio y la comida....,la vista es mejor que el servicio y la comida....
2498,2498,No sé donde debería aparcar para visitar. Enco...,no sé donde debería aparcar para visitar. enco...


In [ ]:
dataset_eval = Dataset.from_pandas(df_eval)
dataset_eval = dataset_eval.map(tokenize_function, batched=True)

# Imprimir las predicciones
predictions = trainer.predict(dataset_eval)
predicted_labels = np.argmax(predictions.predictions, axis=1)
df_eval["predicted_stars"] = predicted_labels

print(df_eval[:10]) 

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

   ID                                             Review  \
0   0  Nos alojamos 5 noches en el Coco Beach Bungalo...   
1   1  Es lindo, pero me hubiera gustado que nos come...   
2   2  El lugar es muy bonito, lo tienen MUY descuida...   
3   3  Además de que Taxco es mágico, el zócalo tiene...   
4   4  Sin duda un lugar Mágico para visitar, debes i...   
5   5  El lugar es un paraíso de la naturaleza , pero...   
6   6  Habíamos leído crítica en Holyeats y dar un pa...   
7   7  Muy agradable el lugar, además está abierto de...   
8   8  Todos los dÃ­as y en especial los fines de sem...   
9   9  Un restaurante informal una manzana de Hidalgo...   

                                              review  predicted_stars  
0  nos alojamos 5 noches en el coco beach bungalo...                3  
1  es lindo, pero me hubiera gustado que nos come...                2  
2  el lugar es muy bonito, lo tienen muy descuida...                2  
3  además de que taxco es mágico, el zócalo tiene..

In [18]:
# Pasar las respuestas en un txt dentro de Drive
for i, pred in enumerate(predicted_labels):
  texto = df_eval.iloc[i]['Review'].replace('\n', ' ').replace('\r', ' ')
  print(f"\n{i} | PREDICCIÓN DE ESTRELLAS: {pred + 1} | {texto}\n")

with open('/content/drive/MyDrive/TripAdvisor datasets/resultados.txt', 'w') as escribir:
    for i, pred in enumerate(predicted_labels):
        texto = df_eval.iloc[i]['Review'].replace('\n', ' ').replace('\r', ' ')
        escribir.write(f"\n{i} | PREDICCIÓN DE ESTRELLAS: {pred + 1} ⭐ | {texto}\n")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
833 | PREDICCIÓN DE ESTRELLAS: 5 | Estuvo increíble. El hotel es hermoso, el cuarto muy agusto con un jacuzzi espectacular y una vista perfecta. La alberca, el servicio a cliente y la comida excelente y riquisima. Regresaría 100%, muy recomendado.


834 | PREDICCIÓN DE ESTRELLAS: 3 | Es un buen lugar para conseguir la auténtica comida local, pero nos quedamos un poco decepcionante en nuestra visita. El relleno era bueno, pero siendo desde el sudoeste, que tan buena en casa si no mejor. El demasido carga el sabor de la sopa de...alubias negras poblano sofritos estaba muy bien hecho y una comida por sí misma. Las enchiladas estaban secos y no tienen suficiente salsa verde en ellos. Realmente quería probar las salsas de mole pero no había ninguno disponible en nuestra visita, una gran decepción. En resumen, no vale la pena volver pero no es el peor lugar probamos en la isla.Más


835 | PREDICCIÓN DE ESTRELLAS: 1 | cuidar a

## Importar el modelo entrenado

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Pruebas manuales

In [44]:
# Pruebas manuales
textos = [
    "La experiencia fue horrible, de lo peor del mundo.",
    "Una experiencia regular, realmente la atención buen buena pero probablemente no volvería a ir.",
    "Muy buen lugar, las cápsulas son muy cómodas y limpias. El servicio en general es muy atento, cuenta con regaderas privadas en cada baño (hombres/mujeres) con shampoo, acondicionador y jabón corporal. Cada locker viene igualmente con toallas y un gancho para ropa.",
    "Las cápsulas están chida pero hay pocos detalles técnicos que necesitan considerar: cuenta con 2 puertos USB pero no son de carga rápida, la Roma de luz es tipo europeo, el reloj está desfasado y los televisores no son inteligentes, no tienen aplicaciones como YouTube. Las cápsulas son cómodas, el lugar es tranquilo, es muy agradable. Los desayunos son sencillos y agradables. La ubicación está retirada de las playas y zonas turísticas"
    ]

resultados = [] # En caso de que se quiera usar, aca se guarda las predicciones

# Imprimir resultados
for i, texto in enumerate(textos):
    # Tokenizar texto
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # Mandar a GPU
    model = model.to("cuda")
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = (torch.argmax(logits, dim=1).item()) + 1 # Para que este en un rango de 1-5
        resultados.append(predicted_class)
        print(f"{i} | PREDICCIÓN DE ESTRELLAS: {predicted_class} ⭐ | {textos[i]}\n")


0 | PREDICCIÓN DE ESTRELLAS: 1 ⭐ | La experiencia fue horrible, de lo peor del mundo.

1 | PREDICCIÓN DE ESTRELLAS: 3 ⭐ | Una experiencia regular, realmente la atención buen buena pero probablemente no volvería a ir.

2 | PREDICCIÓN DE ESTRELLAS: 4 ⭐ | Muy buen lugar, las cápsulas son muy cómodas y limpias. El servicio en general es muy atento, cuenta con regaderas privadas en cada baño (hombres/mujeres) con shampoo, acondicionador y jabón corporal. Cada locker viene igualmente con toallas y un gancho para ropa.

3 | PREDICCIÓN DE ESTRELLAS: 3 ⭐ | Las cápsulas están chida pero hay pocos detalles técnicos que necesitan considerar: cuenta con 2 puertos USB pero no son de carga rápida, la Roma de luz es tipo europeo, el reloj está desfasado y los televisores no son inteligentes, no tienen aplicaciones como YouTube. Las cápsulas son cómodas, el lugar es tranquilo, es muy agradable. Los desayunos son sencillos y agradables. La ubicación está retirada de las playas y zonas turísticas



## Ejemplo para predecir un dataset desde el modelo importado


In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# Cargar modelo desde Drive
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/TripAdvisor datasets/mejor_modelo_beto")
model.eval()

# Cargar dataset sin etiquetas
df_eval = pd.read_excel("/content/drive/MyDrive/TripAdvisor datasets/MeIA_2025_test_two_labels.xlsx")

# Enviar modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Predecir etiquetas
predicted_labels = []
for texto in df_eval["Review"]:
    df_eval["Review"] = df_eval["Review"].apply(limpieza_minima)
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        predicted_labels.append(predicted_class)

# Mostrar predicciones en consola
for i, pred in enumerate(predicted_labels):
    texto = df_eval.iloc[i]['Review'].replace('\n', ' ').replace('\r', ' ')
    print(f"\n{i} | PREDICCIÓN DE ESTRELLAS: {pred + 1} ⭐ | {texto}\n")

# Guardar resultados en archivo .txt en Drive
with open('/content/drive/MyDrive/TripAdvisor datasets/resultados.txt', 'w', encoding='utf-8') as escribir:
    for i, pred in enumerate(predicted_labels):
        texto = df_eval.iloc[i]['Review'].replace('\n', ' ').replace('\r', ' ')
        escribir.write(f"\n{i} | PREDICCIÓN DE ESTRELLAS: {pred + 1} ⭐ | {texto}\n")



0 | PREDICCIÓN DE ESTRELLAS: 4 ⭐ | nos alojamos 5 noches en el coco beach bungalow en tulum en 2016 febrero. el complejo es una verdadera joya, situado en una fantástica playa, y está muy bien cuidado. playa limpian todos los días. los bungalows de la playa son elegantes y limpias, pero no es muy grande, y el cuarto de baño y los aseos son compartidos. los más nuevos bungalows han sido recientemente inaugurado y tienen su propio cuarto de baño y los aseos. el bar de la playa es muy bonita, pero jugar música house durante todo el día con música bass que se pueden oír desde la transat (e incluso de algunos bungalows) que puede ser molesto para algunos huéspedes que buscan un lugar tranquilo para relajarse. el personal del bar y del restaurante es muy agradable y amable, el personal de recepción no es tan bueno, y_x0085_


1 | PREDICCIÓN DE ESTRELLAS: 3 ⭐ | es lindo, pero me hubiera gustado que nos comenten que hay una parte donde dice cuidado, zona de cocodrilo! mi hija se asustó y quis